In [1]:
!apt-get update -y
!apt-get install -y python-xlsxwriter
!pip install openpyxl

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,575 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Err:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
Get:3 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]         
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [909 kB]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Packages [73.8 kB]
Get:

     |████████████████████████████████| 242 kB 4.2 MB/s            


In [7]:
import wandb
import pandas as pd
import datetime
import tqdm

username = "memmelma"
project = "multi_modal" # multi_modal

filter_name = ""
filter_type = "eval"
filter_state = "finished"

metrics = ["eval_metricssuccess", "eval_metricsspl", "eval_metricssoftspl", "eval_metricsdistance_to_goal"]

configs = ["config.VO.REGRESS_MODEL.cls_action", "config.VO.REGRESS_MODEL.visual_backbone", "config.VO.REGRESS_MODEL.pretrain_backbone", "config.VO.REGRESS_MODEL.visual_type"]

out_name = "exp_eval_strip"

In [8]:
# retreive runs

run_ids = []
run_names = []
run_ckpts = []

api = wandb.Api(timeout=45)
runs = api.runs(f"{username}/{project}")

runs_df = pd.DataFrame(columns=["name", "id", "state", *configs, *metrics])

for run in tqdm.tqdm(runs):
    
    if filter_state and run._state != filter_state:
        continue
    
    if filter_type and filter_type not in run.name:
        continue
        
    if filter_name and filter_name not in run.name:
        continue
    
    run_dict = {}
    
    # add base attributes
    run_dict["name"] = run.name.split("[")[0]
    run_dict["id"] = run.id 
    run_dict["state"] = run._state
    
    # add config
    for config in configs:
        # traverse config keys
        run_config_tmp = run.config
        for config_iter in config.split('.'):
            run_config_tmp = run_config_tmp[config_iter]
        run_dict[config] = run_config_tmp
    
    # add filter_operation filter_metric
    run_history = run.history()
    
    # add other metrics corresponding to filter_operation filter_metric
    for metric in metrics:
        # catch models that only evaluate some actions
        if metric in run_history.keys():
            run_dict[metric] = run_history[metric][0]
        else:
            run_dict[metric] = -1
    
    runs_df = runs_df.append(run_dict, ignore_index=True)
    
runs_df = runs_df.sort_values(by=['name'])
runs_df

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:24<00:00,  1.16s/it]


,name,id,state,config.VO.REGRESS_MODEL.cls_action,config.VO.REGRESS_MODEL.visual_backbone,config.VO.REGRESS_MODEL.pretrain_backbone,config.VO.REGRESS_MODEL.visual_type,eval_metricssuccess,eval_metricsspl,eval_metricssoftspl,eval_metricsdistance_to_goal
7,DEP_eval_vit_s_in21k_act_strip_rgb,1hzmuo78,finished,turn_left,small,in21k,"[rgb, depth, top_down_view]",0.258551,0.201477,0.604546,1.489320
18,DEP_eval_vit_s_in21k_rgb_auxd,t74kwk7r,finished,True,small,in21k,"[rgb, depth, top_down_view]",0.119718,0.087151,0.449430,2.608017
17,eval_vit_b_dino_act_strip_d,1gry8ok5,finished,True,base,dino,"[rgb, depth, top_down_view]",0.272636,0.171288,0.327335,2.979022
16,eval_vit_b_dino_act_strip_rgb,2h54l2st,finished,True,base,dino,"[rgb, depth, top_down_view]",0.228370,0.171996,0.562008,1.608402
15,eval_vit_b_in21k_act_strip_d,u3hrsq0p,finished,True,base,in21k,"[rgb, depth, top_down_view]",0.904427,0.691599,0.712599,0.392765
14,eval_vit_b_in21k_act_strip_rgb,2rmiktj4,finished,True,base,in21k,"[rgb, depth, top_down_view]",0.242455,0.181370,0.570564,1.592193
10,eval_vit_b_mmae_act_strip_d,ss7sy48t,finished,True,base,mmae,"[rgb, depth, top_down_view]",0.261569,0.200751,0.587440,1.481957
5,eval_vit_b_mmae_act_strip_d_10,3bychvj0,finished,True,base,mmae,"[rgb, depth, top_down_view]",0.763581,0.581702,0.691170,0.563068
13,eval_vit_b_mmae_act_strip_d_25,1eyaew6f,finished,True,base,mmae,"[rgb, depth, top_down_view]",0.640845,0.493623,0.681124,0.687660
12,eval_vit_b_mmae_act_strip_d_50,dmcm3rfq,finished,True,base,mmae,"[rgb, depth, top_down_view]",0.455734,0.349311,0.644241,1.004521


In [9]:
dt = datetime.datetime.now().strftime("%Y%m%d")
filename = f"{out_name}_{dt}"
filename

'exp_eval_strip_20220506'

In [10]:
# save df as excel file
extension = ".xlsx"
with pd.ExcelWriter(filename+extension) as excel_writer:
    runs_df.to_excel(excel_writer, sheet_name='Sheet1')